# this is a demo to get you more familiar with certain advanced tabular data "tricks" and programming

on 4/7/2024 cell outputs were left to demo nbstripout

In [1]:
import os
import sys
from functools import reduce
import pandas as pd
import numpy as np
sys.path.insert(0, '../../')
from brickstudy import foundation

# We will read in an excel and look for a common key

In [2]:
# Read Excel file with multiple sheets
xls = pd.ExcelFile("../../sample_synthetic_data/example_excel.xlsx")
# Get the list of sheet names
sheet_names = xls.sheet_names
# Print the sheet names
print(sheet_names)

['sheet1', 'sheet2', 'Sheet3row']


In [3]:
excel_file = pd.read_excel("../../sample_synthetic_data/example_excel.xlsx", sheet_name= sheet_names )

In [4]:
print(type(excel_file))


<class 'dict'>


In [5]:
excel_file_default_read = pd.read_excel("../../sample_synthetic_data/example_excel.xlsx")

In [6]:
excel_file_default_read

,row_key,a,b,c
0,PatientA,alpha,bow,wow
1,PatientB,b eta,bow,ijn de bauw
2,PatientC,alpha,bow,bowwow
3,PatientD,b eta,bow,how
4,PatientE,theta,bow,now


So now we made a dictionary of the excel file when we read all parts of it, however, the default reading is only on the first sheet!. Let's access all parts of it by using the better form of reading, with the sheet_name

In [7]:
excel_file

{'sheet1':     row_key      a    b            c
 0  PatientA  alpha  bow          wow
 1  PatientB  b eta  bow  ijn de bauw
 2  PatientC  alpha  bow       bowwow
 3  PatientD  b eta  bow          how
 4  PatientE  theta  bow          now,
 'sheet2':     row_key  blank  na  blanky    d      e      f
 0  PatientA    NaN NaN     NaN   do  shcmu     do
 1  PatientB    NaN NaN     NaN   do     do  schmu
 2  PatientC    NaN NaN     NaN   do     do     do
 3  PatientD    NaN NaN     NaN  it     it     it 
 4  PatientE    NaN NaN     NaN  now    now    now,
 'Sheet3row':    fg  h   row_key
 0   4  2  PatientA
 1   5  4  PatientB
 2   7  3  PatientC
 3   8  3  PatientD
 4   9  3  PatientE}

In [8]:
len(sheet_names)

3

In [9]:
sheet1 = excel_file[sheet_names[0]]
sheet1

,row_key,a,b,c
0,PatientA,alpha,bow,wow
1,PatientB,b eta,bow,ijn de bauw
2,PatientC,alpha,bow,bowwow
3,PatientD,b eta,bow,how
4,PatientE,theta,bow,now


In [10]:
sheet2 = excel_file[sheet_names[1]]
sheet3 = excel_file[sheet_names[1]]


In [11]:
sumsheet = sheet1.merge(sheet2, on='row_key')
sumsheet

,row_key,a,b,c,blank,na,blanky,d,e,f
0,PatientA,alpha,bow,wow,NaN,NaN,NaN,do,shcmu,do
1,PatientB,b eta,bow,ijn de bauw,NaN,NaN,NaN,do,do,schmu
2,PatientC,alpha,bow,bowwow,NaN,NaN,NaN,do,do,do
3,PatientD,b eta,bow,how,NaN,NaN,NaN,it,it,it
4,PatientE,theta,bow,now,NaN,NaN,NaN,now,now,now


In [12]:
sheet3

,row_key,blank,na,blanky,d,e,f
0,PatientA,NaN,NaN,NaN,do,shcmu,do
1,PatientB,NaN,NaN,NaN,do,do,schmu
2,PatientC,NaN,NaN,NaN,do,do,do
3,PatientD,NaN,NaN,NaN,it,it,it
4,PatientE,NaN,NaN,NaN,now,now,now


In [13]:
# But we can do it programmatically!

In [14]:
def find_my_key(excel_book_name):
    # Read Excel file with multiple sheets
    xls = pd.ExcelFile(excel_book_name)
    # Get the list of sheet names
    sheet_names = xls.sheet_names
    # read in the excel 
    excel_file = pd.read_excel(excel_book_name, sheet_name= sheet_names)
    sheet_list = []
    for sheet_name in sheet_names:
        sheet = excel_file[sheet_name]
        sheet_list.append(sheet)
    set_columns = []
    for listi in sheet_list:
        setcolumns_set = set(listi.columns)
        set_columns.append(setcolumns_set)
    result_set = set_columns[0]    
    for set_specific in set_columns[1:]:
        common_columns = result_set.intersection(set_specific)
        common_columns = list(common_columns)
    
    return common_columns

In [15]:
listicle = find_my_key("../../sample_synthetic_data/example_excel.xlsx")

let's call it from our  budding lirbary

In [16]:
listicle = foundation.find_my_key("../../sample_synthetic_data/example_excel.xlsx")
listicle

['row_key']

## now we need to know something about the shape of each sheet

In [17]:
def compare_sheets(excel_book_name):
    """
    This function reads a multisheet excel
    and prints back some infor you want to know before merging
    """
    # Read Excel file with multiple sheets
    xls = pd.ExcelFile(excel_book_name)
    # Get the list of sheet names
    sheet_names = xls.sheet_names
    # read in the excel 
    excel_file = pd.read_excel(excel_book_name, sheet_name= sheet_names)
    sheet_row_numbers = []
    sheets = []
    for sheet_name in sheet_names:
        sheet = excel_file[sheet_name]
        sheets.append((sheet_name ,sheet))
        sheet_row_numbers.append(sheet.shape[0])
    first_one = sheet_row_numbers[0]
    counter = 1
    for element in sheet_row_numbers[1:]:
        if first_one == element:
            counter +=1
    if counter == len(sheet_row_numbers):
        print("Your sheets all have the same number of rows")
    else:
        print("Your sheets doe not all have the same number of rows")
        print("Row numbers are", sheet_row_numbers)
    for sheety in sheets:
        if sheety[1].isna().sum().sum() > 0:
            print("You have some nans in", sheety[0])
            print(sheety[1].isna().sum())
            
    return None

In [18]:
foundation.compare_sheets("../../sample_synthetic_data/example_excel_uneven.xlsx")

Your sheets doe not all have the same number of rows
Row numbers are [6, 5, 5]
You have some nans in sheet2
row_key    0
blank      5
na         5
blanky     5
d          0
e          0
f          0
dtype: int64
